In [9]:
import re
import pandas as pd
from pathlib import Path
from collections import defaultdict
import unicodedata


In [10]:
MAPA_ASIGNATURAS = {
    "MATEMATICAS": "MAT",
    "MAT-AV": "MAT",
    "MAT": "MAT",
    "MAT-APL": "MAT",
    "MATES": "MAT",
    "MATEMATICAS APLICADAS": "MAT",
    "MATEMATICAS AVANZADAS": "MAT",
    "LENGUA": "LEN",
    "LENGUA CASTELLANA": "LEN",
    "LENGUA CASTELLANA Y LITERATURA": "LEN",
    "INGLES": "ING",
    "ENGLISH": "ING",
    "LENGUA EXTRANJERA INGLES": "ING",
    # FÍSICA Y QUÍMICA
    "FISICA Y QUIMICA": "FYQ",
    "FISICAYQUIMICA": "FYQ",
    "FYQ": "FYQ",
    "FISICA": "FISICA",
    "FIS": "FISICA",
    "QUIMICA": "QUIMICA",
    "QUI": "QUIMICA",
    "BIOLOGIA": "BIO",
    "BIO Y GEO": "BIO",
    "BYG": "BIO",
    "BIO": "BIO",
    "BIOLOGIA Y GEOLOGIA": "BIO",
    "LATIN": "LAT",
    "HISTORIA": "HIST",
    "HISTORIA DEL ARTE": "HISTART",
    "GEOGRAFIA": "GEO",
    "GEOGRAFIA E HISTORIA":"GEH",
    # EDUCACIÓN FÍSICA
    "EDUCACION FISICA": "EF",
    "ED FISICA": "EF",
    "EF": "EF",
    "EDUCACION PLASTICA": "PLAS",
    "PLASTICA": "PLAS",
    "EDUCACION PLASTICA Y VISUAL": "PLAS",
    "EPV": "PLAS",
    "PLAS": "PLAS",
    "FRANCES": "FR",
    "SEGUNDO IDIOMA EXTRANJERO": "FR",
    "FR": "FR",
    "MUSICA":"MUS",
    "HISTORIA DE LA MUSICA": "MUS",
    "HISTORIA DE LA MUSICA Y LA DANZA": "MUS",
    "HISTORIA Y CULTURA DEL ARTE": "HISTART",
    "HISTORIA DEL MUNDO CONTEMPORANEO": "HIST",
    "HISTORIA DE ESPAÑA": "HIST",
    "LITERATURA UNIVERSAL": "LITUNI",
    "LIT UNI": "LITUNI",
    # ECONOMÍA
    "ECONOMIA": "ECO",
    "ECO": "ECO",
    # TECNOLOGÍA
    "TECNOLOGIA": "TEC",
    "TECNO": "TEC",
    "TEC": "TEC",
    # FILOSOFÍA
    "FILOSOFIA": "FIL",
    "FIL": "FIL",
    "AUXILIAR": "AUX",
    "APOYO": "AUX",
    "AUX": "AUX",
    # RELIGIÓN / ORIENTACIÓN / APOYO
    "RELIGION": "REL",
    "ORIENTACION": "ORIENT",
    "VALORES ETICOS": "ORIENT",
    "REL": "REL",
    "AUDIO VISUALES": "AUD",
    "AUD": "AUD",
    "A-V": "AUD"
}

MAPA_CURSOS_TEXTO = {
    "1 ESO": "1E",
    "1ESO": "1E",
    "2 ESO": "2E",
    "2ESO": "2E",
    "3 ESO": "3E",
    "3ESO": "3E",
    "4 ESO": "4E",
    "4ESO": "4E",
    "1 BACH": "1BACH",
    "1BACHILLERATO": "1BACH",
    "2 BACH": "2BACH",
    "2BACHILLERATO": "2BACH",
}


In [11]:
def normalizar(texto: str) -> str:
    texto = texto.upper()
    texto = unicodedata.normalize("NFD", texto)
    texto = texto.encode("ascii", "ignore").decode("utf-8")
    return texto.strip()


In [12]:

PATRON_HORAS_ESTRICTO = re.compile(
    r"""
    (?P<curso>
        1E|2E|3E|4E|
        1ESO|2ESO|3ESO|4ESO|
        1BACH|2BACH
    )
    \s+
    (?P<asig>[A-Z\s\-]+?)
    \s+
    (?P<horas>\d+)
    """,
    re.VERBOSE
)




In [13]:
def normalizar_asignatura(asig_raw: str) -> str:
    # asig_raw YA viene normalizada
    return MAPA_ASIGNATURAS.get(asig_raw.strip(), asig_raw.strip())



In [14]:
def normalizar_curso(texto: str) -> str:
    texto = texto.strip()
    for k, v in MAPA_CURSOS_TEXTO.items():
        if k in texto:
            return v
    return texto 

    

In [15]:

def parse_horas_txt(ruta_txt: str) -> pd.DataFrame:
    filas = []

    with open(ruta_txt, encoding="utf-8") as f:
        for linea in f:
            linea = normalizar(linea)
            if not linea:
                continue

            m = PATRON_HORAS_ESTRICTO.match(linea)
            if not m:
                continue  # línea inválida → se ignora

            filas.append({
                "curso": normalizar_curso(m.group("curso")),
                "asignatura": normalizar_asignatura(m.group("asig")),
                "horas_semanales": int(m.group("horas"))
            })

    df = pd.DataFrame(filas)

    # Unificar posibles duplicados
    df = (
        df.groupby(["curso", "asignatura"], as_index=False)
          .agg({"horas_semanales": "max"})
    )

    return df



In [16]:
df_horas = parse_horas_txt("../data/horas_semanales.txt")
df_horas.to_csv("../data/prueba_horas_semanales.csv", index=False)
